In [20]:
a = ["groom_list","lab_list","cinema_list","piano_list","barb_list",
 "military_list","none_list","vestment_list","curtain_list",
 "college_list","male_dress_list","female_dress_list"]

#Note:

# Affinity propagation can be viewed as a spectral clustering algorithm that requires each cluster to vote 
# for a good exemplar from within its data points. Hierarchical agglomerative clustering starts with every 
# data point as its own cluster and then recursively merges pairs of clusters, but that method makes hard 
# decisions that can cause it to get stuck in poor solutions. Affinity propagation can be viewed as a 
# version of hierarchical clustering that makes soft decisions so that it is free to hedge its bets when 
# forming clusters.

#what you used: affinity propogation  An algorithm that identifies exemplars among data points and forms clusters of data points around these exemplars. ... It is similar to k-means clustering or the EM algorithm, except that the centers must lie on data points
#affinity propogation is literally like a fancier k-means. But it doesn't give relative distances between the clusters which is annoying

#total number of possibilities for each picture vs picture comparison (1024 choose 5 + 1024 choose 4 + 1024 choose 3 + 1024 choose 2 + 1024 choose 1 + 1024 choose 0)
#because for 2 given pictures they can have 0-5 objects in common and those 0-5 objects are chosen from 1024 possible objects
 
#Due to this infinite number of possibilities.... hierarchical clustering no longer made sense

#maybe I need to do a data reduction on object clustering. Like if a photo is labelled as a chair should that be in a different category than a photo labelled as sitting?

#affinity propogation clusters the photos by how similar or how many labels the photos have in common.
#The results I get from this clustering doesn't necessarily imply that the photos have the same labels in common.
#it means that if a photo is in a cluster, it shares more labels in common with this subset of photos than it does with any other cluster of photos.
#This is useful if you say: "I want a subset from the teenie harris archive that contains similar things."
#This is also useful if you want to differentiate between photos with bow ties grooms and vestments vs piano bow ties grooms and vestments

# if all the photos are equidistant from eachother in space then it's almost like the exemplars for the clusters are randomly chosen
# and although the clusters are still useful for the fact that they can be represented by this exemplar and are similar to it. it's less useful

#I have 2 options merge the subsets together with my own algorithm, or leave the clusters as they are.
# Cannot re run the algorithm and lower the preferences to get less clusters because preferences is already set at its lowest value
# preferences is set to the median similarity value in the similarity matrix which is 0 in this case because the similarity matrix is so sparse.

#And you can do a search for subsets with particular labels assigned to them, and you can do that search through the results of hiearchical clustering

#This means it's really important to get all of the labels that appear in the cluster and the frequency counts of each label
#you already have the top 5 labels from each photo.

# The reason why k-nearest neighbors wasn't chosen is because: A) we don't know k and k is subjective anyways.
# B) K-nearest neighbors works well for smaller search spaces. i.e. 3 dimensions or less. since we have 1024 dimensions it'd be unlikely that we'd find correct centers so the clusters wouldn't mean anything
#Hierarchical clustering, clusters the photos by what labels the photos have in common. 2^1024 possibilities for position in hiearchical clustering.


### Task 1 for this script. For each cluster you'd like to find the photo that is the exemplar for the cluster, 
## what the labels in the cluster are and 
## a frequency chart of the labels containing the percentage of the photos in the cluster that each label shows up in.
## This data should all be put into mongo as well. This means you need a new collection called Clusters in addition to Faces and Photos.
## each cluster will contain the three things listed above previously.
## In theory you could use the frequency table of labels to get distances between clusters and reduce them down some more
## So some of these clusters are trash and only related to eachother by having one thing in common 
## and a set of things that they all exclude which pushes them together in euclidean space
## assign a quality metric to clusters


## Task 2 insert the top 5 image classification labels for each photo

import pickle
import os
from pymongo import MongoClient
import numpy as np
import numpy.matlib

client = MongoClient('localhost:27017')
db = client.Teenie

#labels_lab_list.pickle is zero bytes and affinityPropResults_groom_list.pickle is zero bytes
centers,PhotoClusterDict = None, None
top5 = None
with open('top5perphotoDictionary.pickle', 'rb') as handle:
    top5 = pickle.load(handle)
quality_scores_ = [0]*40
for tag in a: 
    
    print tag  
    all_photos_in_tag = np.load(tag+".npy")
    with open('ClusterCenters/affinityPropResults_'+tag+'.pickle', 'rb') as handle2:
        centers = pickle.load(handle2)

    with open('Clusters/ClusterPhotoMapping_'+tag+'.pickle', 'rb') as handle3:
        PhotoClusterDict = pickle.load(handle3)

    for i in xrange(len(PhotoClusterDict.keys())):
        new_data_point = {}
        new_data_point["name"] = tag+str(i)
        new_data_point["tag"] = tag
        try:
            new_data_point["exemplar"] = db.Photos.find_one({"path" : {"$regex" : ".*/" + all_photos_in_tag[centers[i]][:-5] + ".png"}})["_id"]
        except:
            print "error1", tag, all_photos_in_tag[centers[i]][:-5]
        new_data_point["photos"] = []
        photos = PhotoClusterDict[i+1]
        size = len(photos)
        new_data_point["size"] = size
        if size < 1 : continue
        for i in photos:
            try:
                new_id = db.Photos.find_one({"path" : {"$regex" : ".*/" + i[:-5] + ".png"}})["_id"]
                new_data_point["photos"].append(new_id)
            except:
                print "error2", tag+str(i), i[:-5]
        cluster_freq = {}
        quality_score = 0 
        # each label that is in more than 50% of the photos adds to the score
        # every additional 7 photos adds to the score
        # clusters with less than 7 photos get a quality score of zero
        if len(new_data_point["photos"]) == 0: continue
        #Build Frequency Table for each cluster
        for filename in photos:
            p_labels = top5[filename]
            try:
                idd = db.Photos.find_one({"path" : {"$regex" : ".*/" + filename[:-5] + ".png"}})["_id"]
                db.Photos.update({'_id': idd},{'$set': {'Top5ObjClassifications': list(p_labels)}}, upsert=False, multi=False)
            except:
                print "error3", tag+str(i), filename[:-5] , filename, type(filename), len(filename), "done"
            for label_ in p_labels: 
                if label_ in cluster_freq.keys():
                    cluster_freq[label_] += 1
                else:
                    cluster_freq[label_] = 1 
        
        #Assign Quality Score for each cluster. This was purely subjective weighting. 
        # this metric gives me clusters that have a good amount of tags and a decent size according to my perception

        for lab in cluster_freq:
            if cluster_freq[lab]*1.0/size > 0.5 : quality_score +=3
        quality_score += size/5
        if size < 8 : quality_score = 0
            
        quality_scores_[quality_score] += 1
        new_data_point["frequencies"] = cluster_freq               
        new_data_point["quality_score"] = quality_score
        db.Clusters.insert(new_data_point)
#         if quality_score > 18: print cluster_name, center, photos, size, cluster_freq, quality_score

#     print tag, quality_scores_ #shows the distribution for quality clusters for each tag

groom_list
error2 groom_list9875.jpeg 9875
error3 groom_list9875.jpeg 9875 9875.jpeg <type 'numpy.string_'> 9 done
error2 groom_list9872.jpeg 9872
error3 groom_list9872.jpeg 9872 9872.jpeg <type 'numpy.string_'> 9 done
lab_list
error2 lab_list9857.jpeg 9857
error3 lab_list9857.jpeg 9857 9857.jpeg <type 'numpy.string_'> 9 done
cinema_list
piano_list
barb_list
military_list
none_list
vestment_list
curtain_list
college_list
male_dress_list
female_dress_list


In [ ]:
quality_scores_

In [ ]:
quality_scores_

In [ ]:
'45762.jpeg'[:-5]

In [ ]:
q= []
q.append(5)
q.append(6)
q

In [18]:
db.Photos.update({'_id': idd},{'$set': {'Top5ObjClassifications': list(p_labels)}}, upsert=False, multi=False)

{u'n': 1, u'nModified': 1, u'ok': 1, 'updatedExisting': True}

In [17]:
type(list(p_labels))

list